<br><strong>Coding for Venezuela - Team Penguin</strong>
<br><strong>Track 2 - Dashboarding for NGOs</strong>

Team Penguin: Duy Pham, Linh Le, Paolo Schirru, Peter Huesmann, Skander Driss <br>

<h3>Instructions</h3><br>

This program is using bigquery raw data tables in Angostura to create a dataframe that transformed through data cleaning and feature engineering in Python and visualised in Tableau. The Python script can be run on Airflow and is available on github and devpost.
After running the python script a csv file is created that has to be imported into Tableu. 
The Python script is soft-coded and does not require changes if additional data is added.

<h4>Instructions on cleaning the data</h4><br>
    
The data is cleaned automatically when the script is run and saved as a new local csv file.
The data is cleaned in multiple steps:
    1. Creating Missing Value columns
    2. Changing the report date formating
    3. Imputing values used for inconsistency analysis
    4. Creating logical inconsistency analysis for data quality
    5. Creating a strike free column
    6. Creating a duplicate indicator column for data quality
    7. Removing duplicates from dataset
    8. Imputing missing values
    9. Creating missing values percentage for data quality
    10. Ordinal encoding columns for Tableau
    11. Exporting to csv

<h4><strong>Instructions on generating dashboards based on the scripts</strong><br>
 

<h3>Data Exploration</h3><br>

<strong>Domain Research</strong><br>

Since 2013, Venezuela has been experiencing a humanitarian crisis as a result of socioeconomic and political tensions. This has caused widespread poverty and massive shortages of basic necessities, including food and healthcare. National blackouts and power failures, as well as vaccine, equipment, and medication shortages have caused mortality rates to soar. Over 4 million people, or about 10% of Venezuela’s total population, have fled the country. About 30% of medical professionals leave the country, particularly young doctors. 
Government data is virtually non-existent, which is where the Angostura Data Platform comes in. This is a data collection and storage platform, allowing hospital personnel across the region to input data weekly, and NGOs focusing on the Venezuelan crisis to make effective, data-driven decisions.<br><br>

<strong>Data Exploration</strong><br><br>
Looking for variables related to supply, equipment, and operability as key factors for NGOs. The NGOs require  data to decide on how to allocate their budget and resources. They need information on the requirements and potential impact for each hospital and region. They need to be able to filter technical topics in the dashboards to allocate specific budgets for different solutions, such as the needs for specific medicine and equipment. They also need to be able to filter for time, hospital and region. After exploring the data we decided to focus on the scarcity of supplies and equipment, operability of services and data quality. Descriptions are given in the comments on feature engineering.

<h3>Setup</h3><br>

Setup and data loading for the data cleaning and feature engineering

In [1]:
################################################################################
# Import Packages
################################################################################

import pandas as pd                                   # data science essentials
import matplotlib.pyplot as plt                       # essential graphical output                                 # enhanced graphical output
import statsmodels.formula.api as smf                 # regression modeling
import numpy as np                                    # Math
from scipy import stats
import seaborn
import time

# setting options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%matplotlib inline

<strong>Authentication Setup</strong>

Before being able to query the Angostura database you need to authenticate.
You will have read-only access to the tables.

In [2]:
# First we need to get the JSON key for authentication

import urllib.request

url = 'https://storage.googleapis.com/angostura-public/hult-hackathon-key.json'
urllib.request.urlretrieve(url, './hult-hackathon-key.json')

# Second, we add the key to our environment

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './hult-hackathon-key.json'

<strong>Import GCloud API</strong>

In [3]:
from google.cloud import bigquery

client = bigquery.Client()

<strong>Query</strong>

In [4]:
# We can dump the BigQuery results directly into a DataFrame
QUERY = ('select * from `angostura_dev`.eh_health_survey_response')

df = client.query(QUERY).to_dataframe()

In [7]:
df.head()

,timestamp,report_week,operability_icu,power_outage_equipment_failure_specify,power_generator_available,power_outage_mortatility,power_outage_deaths_count,mortality_hospital_failure_cardiovascular_count,mortality_hospital_failure_cardiovascular_cause,mortality_hospital_failure_trauma_count,mortality_hospital_failure_trauma_cause,strike_medical_staff_affected,strike_nurses_affected,operability_icu_p,strike_other_staff_affected,strike_patients_affected,strike_other_affected,staff_violence_affected_reasons,epidemiological_emergency_suspected_diseases,rrt_avail_high_flow_catheters,rrt_avail_blood_tests_hiv_hvb_hvc_vdr,rrt_avail_immediate_access_urea_reduction_bun,nCoV_face_mask_avail,nCoV_respiratory_isolation_protocol_avail,operability_er,nCoV_isolation_area_avail,operability_sx,operability_lab,operability_uls,operability_ct_mri,operability_xr,er_avail_adrenalin,er_avail_atropine,hospital_code,er_avail_dopamine,er_avail_cephalosporins_betalactams,er_avail_aminoglycosides_quinolone,er_avail_vancomycin_clindamycin,er_avail_lidocaine,er_avail_minor_opioids,er_avail_major_opioids,er_avail_iv_fluids,er_avail_diazepam_dph,er_avail_heparin,federal_entity,er_avail_steroids,er_avail_insulin,er_avail_asthma,er_avail_blood_pressure,er_avail_defibrillator,er_avail_ott_intubation,er_avail_catheter,er_avail_oxygen_suction,sx_avail_minor_opioids,sx_avail_major_opioids,hospital_type,sx_avail_anesthetic_gases,sx_avail_anesthetics_iv,sx_avail_relaxants,sx_avail_ott_intubation,sx_avail_patient_lingerie_kit,sx_avail_disposables_mask_gloves_gown,sx_avail_oxygen_suction,rrt_avail,rrt_operability,rrt_date_stopped_operability,administrative_entity,rrt_reason_stopped_operability,rrt_avg_daily_patients,rrt_peritoneal_count,rrt_peritoneal_reason_not_performed,rrt_num_hemodialysis,rrt_num_hemodialysis_acute,rrt_num_hemodialysis_chronic,rrt_num_hemodialysis_equipments,rrt_num_hemodialysis_equipments_operability,rrt_hemodialysis_avail_filter,arch_beds_count,rrt_hemodialysis_avail_lines,rrt_hemodialysis_avail_kit_hemodialysis,rrt_hemodialysis_avail_iron,rrt_hemodialysis_avail_b_complex,rrt_hemodialysis_avail_calcium,rrt_hemodialysis_avail_zemblar,rrt_reverse_osmosis_unit_operability,rrt_staff_nephrology,rrt_staff_md,rrt_staff_resident,op_beds_count,rrt_staff_nurse,rrt_staff_nurse_nephrologist,nutr_avail,nutr_operability,nutr_date_stopped_operability,nutr_reason_stopped_operability,nutr_num,nutr_daily_freq_meal,nutr_quality,nutr_freq_milk_formulas,op_beds_er_count,pneumonia_antibiotic_therapy_latency,mi_thrombolytic_treatment_latency,er_staff_residents_and_rural_day_on_call,er_staff_specialist_day_on_call,er_staff_mic_day_on_call,er_staff_nurse_day_on_call,er_staff_non_professional_nurse_day_on_call,er_staff_residents_and_rural_night_on_call,er_staff_specialist_night_on_call,er_staff_mic_night_on_call,op_pavilions_count,er_staff_nurse_night_on_call,er_staff_non_professional_nurse_night_on_call,wash_failure_icu,wash_failure_er,wahs_failure_sx,power_outage,power_outage_avg_failures_per_day,power_outage_days_count,power_outage_avg_duration,power_outage_equipment_failure
0,2019-11-29T19:23:06.312Z,47 del 2019,Todos los días,,Hay pero no funciona,No,NaN,1.0,"Equipos, Medicamentos, Recurso Humano, Resto e...",0.0,,,,Nunca ha existido,,,,"Robos , hurtos o disparos dentro del centro as...",TBC en personal de salud,,,,,,Todos los días,,Todos los días,No operativa,No operativa,Nunca ha existido,No operativa,No hubo,No hubo,AMA000,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Entre 1 y 2 días,Amazonas,No hubo,Todos los días,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Tipo II,Todos los días,Todos los días,Todos los días,Todos los días,Todos los días,No hubo,Todos los días,Sí,No,Mas de 1 año,MPPS,"Falta de material quirúrgico, Falta de insumos...",NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,170,,,,,,,,,,,42,,,Sí,Sí,,,Todos los días,Se sirven menos de 3 comidas al día,No se siguen recomendaciones del especialista ...,Hubo fórmulas lácteas entr

In [8]:
df.isnull().sum()

timestamp                                            0
report_week                                          0
operability_icu                                      0
power_outage_equipment_failure_specify               0
power_generator_available                            0
power_outage_mortatility                             0
power_outage_deaths_count                          560
mortality_hospital_failure_cardiovascular_count    348
mortality_hospital_failure_cardiovascular_cause      0
mortality_hospital_failure_trauma_count            390
mortality_hospital_failure_trauma_cause              0
strike_medical_staff_affected                        0
strike_nurses_affected                               0
operability_icu_p                                    0
strike_other_staff_affected                          0
strike_patients_affected                             0
strike_other_affected                                0
staff_violence_affected_reasons                      0
epidemiolo

In [5]:
################################################################################
# Feature Engineering 
################################################################################

# looping over columns with missing values
for col in df:

    # creating columns with 1s if missing and 0 if not
    if df[col].isnull().astype(int).sum() > 0:
        df['m_'+col] = df[col].isnull().astype(int)


<h3>Data Cleaning and Feature Engineering</h3><br>


<h4>Creating missing value flags</h4><br>
We start by flagging all the missing values and creating missing value columns as a binary 1/0 if a column has a missing value.

<h4>Changing report date</h4><br>
Changing report week into a more readable format and setting up duplicate removal


In [6]:
################################################################################
# Support Functions
################################################################################

# Changing Report week to date time column
def week_to_datetime (data, col):
    report_date = data[col].str.replace(' del ', '_')
    data['report_date'] = pd.to_datetime('0_'+ report_date ,format='%w_%W_%Y', dayfirst='True')
    return data

# Dropping duplicates
def later_duplicated_hospital (data):
    data.drop_duplicates(['report_date', 'hospital_code'],keep='last',inplace = True)
    return data

In [7]:
# Using Week to date time function
week_to_datetime (df, 'report_week')

,timestamp,report_week,operability_icu,power_outage_equipment_failure_specify,power_generator_available,power_outage_mortatility,power_outage_deaths_count,mortality_hospital_failure_cardiovascular_count,mortality_hospital_failure_cardiovascular_cause,mortality_hospital_failure_trauma_count,mortality_hospital_failure_trauma_cause,strike_medical_staff_affected,strike_nurses_affected,operability_icu_p,strike_other_staff_affected,strike_patients_affected,strike_other_affected,staff_violence_affected_reasons,epidemiological_emergency_suspected_diseases,rrt_avail_high_flow_catheters,rrt_avail_blood_tests_hiv_hvb_hvc_vdr,rrt_avail_immediate_access_urea_reduction_bun,nCoV_face_mask_avail,nCoV_respiratory_isolation_protocol_avail,operability_er,nCoV_isolation_area_avail,operability_sx,operability_lab,operability_uls,operability_ct_mri,operability_xr,er_avail_adrenalin,er_avail_atropine,hospital_code,er_avail_dopamine,er_avail_cephalosporins_betalactams,er_avail_aminoglycosides_quinolone,er_avail_vancomycin_clindamycin,er_avail_lidocaine,er_avail_minor_opioids,er_avail_major_opioids,er_avail_iv_fluids,er_avail_diazepam_dph,er_avail_heparin,federal_entity,er_avail_steroids,er_avail_insulin,er_avail_asthma,er_avail_blood_pressure,er_avail_defibrillator,er_avail_ott_intubation,er_avail_catheter,er_avail_oxygen_suction,sx_avail_minor_opioids,sx_avail_major_opioids,hospital_type,sx_avail_anesthetic_gases,sx_avail_anesthetics_iv,sx_avail_relaxants,sx_avail_ott_intubation,sx_avail_patient_lingerie_kit,sx_avail_disposables_mask_gloves_gown,sx_avail_oxygen_suction,rrt_avail,rrt_operability,rrt_date_stopped_operability,administrative_entity,rrt_reason_stopped_operability,rrt_avg_daily_patients,rrt_peritoneal_count,rrt_peritoneal_reason_not_performed,rrt_num_hemodialysis,rrt_num_hemodialysis_acute,rrt_num_hemodialysis_chronic,rrt_num_hemodialysis_equipments,rrt_num_hemodialysis_equipments_operability,rrt_hemodialysis_avail_filter,arch_beds_count,rrt_hemodialysis_avail_lines,rrt_hemodialysis_avail_kit_hemodialysis,rrt_hemodialysis_avail_iron,rrt_hemodialysis_avail_b_complex,rrt_hemodialysis_avail_calcium,rrt_hemodialysis_avail_zemblar,rrt_reverse_osmosis_unit_operability,rrt_staff_nephrology,rrt_staff_md,rrt_staff_resident,op_beds_count,rrt_staff_nurse,rrt_staff_nurse_nephrologist,nutr_avail,nutr_operability,nutr_date_stopped_operability,nutr_reason_stopped_operability,nutr_num,nutr_daily_freq_meal,nutr_quality,nutr_freq_milk_formulas,op_beds_er_count,pneumonia_antibiotic_therapy_latency,mi_thrombolytic_treatment_latency,er_staff_residents_and_rural_day_on_call,er_staff_specialist_day_on_call,er_staff_mic_day_on_call,er_staff_nurse_day_on_call,er_staff_non_professional_nurse_day_on_call,er_staff_residents_and_rural_night_on_call,er_staff_specialist_night_on_call,er_staff_mic_night_on_call,op_pavilions_count,er_staff_nurse_night_on_call,er_staff_non_professional_nurse_night_on_call,wash_failure_icu,wash_failure_er,wahs_failure_sx,power_outage,power_outage_avg_failures_per_day,power_outage_days_count,power_outage_avg_duration,power_outage_equipment_failure,m_power_outage_deaths_count,m_mortality_hospital_failure_cardiovascular_count,m_mortality_hospital_failure_trauma_count,m_rrt_avg_daily_patients,m_rrt_peritoneal_count,m_rrt_num_hemodialysis,m_rrt_num_hemodialysis_acute,m_rrt_num_hemodialysis_chronic,m_rrt_num_hemodialysis_equipments,m_rrt_num_hemodialysis_equipments_operability,report_date
0,2019-08-20T22:45:52.803Z,32 del 2019,Todos los días,,Funciona todos los días,No,NaN,2.0,"Equipos, Medicamentos, Recurso Humano, Resto e...",NaN,,,,Nunca ha existido,,,,,,,,,,,Todos los días,,Todos los días,Entre 3 y 5 días,No operativa,Nunca ha existido,No operativa,No hubo,No hubo,AMA000,No hubo,No hubo,No hubo,No hubo,Entre 1 y 2 días,No hubo,No hubo,No hubo,No hubo,No hubo,Amazonas,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Entre 1 y 2 días,No hubo,No hubo,Tipo II,Todos los días,Todos los días,Todos los días,Todos los días,Entre 1 y 2 días,Entr

<h4>Imputing values for columns required in the inconsistency analysis</h4><br>

The Inconsistency 

In [8]:
# Counting number of observation before removing duplicates and saving it for use in
# inconsistency analysis
lenght1 = len(df)

In [9]:
# Imputing 'mortality_hospital_failure_cardiovascular_count'
# We can assume that most hospitals without cases simply left this field blank and can therfore imput
# '0'
# This column and mortality_hospital_failure_cardiovascular_cause are correlated. However in some cases
# a cause of mortality is listed but no count is given. This most likely is due to incorrect data
# gathering. We could assume a number of 1 in this case or leave it at zero. 
fill = 0
df['mortality_hospital_failure_cardiovascular_count'] = df['mortality_hospital_failure_cardiovascular_count'].fillna(fill)

In [10]:
# Imputing 'mortality_hospital_failure_cardiovascular_cause'
# If there was a count we can impute unkown, if there is no count we can impute no deaths or 0
fill = 'unknown'
df['mortality_hospital_failure_cardiovascular_cause'] = df['mortality_hospital_failure_cardiovascular_cause'].fillna(fill)

In [11]:
# Imputing 'mortality_hospital_failure_trauma_count'
# We can assume that most hospitals without cases simply left this field blank and can therfore imput
# '0'
# This column and mortality_hospital_failure_trauma_cause are correlated. However in some cases
# a cause of mortality is listed but no count is given. This most likely is due to incorrect data
# gathering. We could assume a number of 1 in this case or leave it at zero. 
fill = 0
df['mortality_hospital_failure_trauma_count'] = df['mortality_hospital_failure_trauma_count'].fillna(fill)

In [12]:
# Imputing 'mortality_hospital_failure_trauma_cause'
# If there was a count we can impute unkown, if there is no count we can impute no deaths or 0
fill = 'unknown'
df['mortality_hospital_failure_trauma_cause'] = df['mortality_hospital_failure_trauma_cause'].fillna(fill)

<strong>Mortality rates and causes</strong><br><br>
Mortality rates and causes have a number of inconsistencies. We can create an additional columns for misreported values, where a cause of mortality is given but the count is either blank or 0. We cannot assume that the count in these cases would be one or more, but it is unlikely that it is zero since, the cause of mortality is a write in field, that requires more effort than the count. It is unlikely staff would fill the cause column if no patient had died.

In [13]:
x=0
placholder_lst =[]
while x < lenght1 :
    if df.loc[x, 'mortality_hospital_failure_trauma_cause'] != 'unknown' and df.loc[x, 'mortality_hospital_failure_trauma_count'] < 1:
        placholder_lst.append(1)
       
    elif df.loc[x, 'mortality_hospital_failure_trauma_cause'] != 'unknown' and df.loc[x, 'mortality_hospital_failure_trauma_count'] > 0:
            placholder_lst.append(0)
    else:
        placholder_lst.append(0)
    x += 1


In [14]:
x=0
placholder_lst2 =[]
while x < lenght1 :
    if df.loc[x, 'mortality_hospital_failure_cardiovascular_cause'] != 'unknown' and df.loc[x, 'mortality_hospital_failure_cardiovascular_count'] < 1:
        placholder_lst2.append(1)
       
    elif df.loc[x, 'mortality_hospital_failure_cardiovascular_cause'] != 'unknown' and df.loc[x, 'mortality_hospital_failure_cardiovascular_count'] > 0:
            placholder_lst2.append(0)
    else:
        placholder_lst2.append(0)
    x += 1


In [15]:
# Imputing 'power_outage_mortality '
# Very inconsistent data. Only 10 observations with power outage mortality yes and some of those
# with power outage mortality yes have no power outage death counts
# The remaining ones with power outage mortality no will be imputed as 0
def fill_na_power_mortality (data): 
    data.loc[(data['power_outage_deaths_count'].isnull()) & (data['power_outage_mortatility']=='No'), 'power_outage_deaths_count'] = 0
    return data
fill_na_power_mortality(df)
fill=0
df['power_outage_deaths_count'] = df['power_outage_deaths_count'].fillna(fill)

In [16]:
x=0
placholder_lst3 =[]
while x < lenght1 :
    if df.loc[x, 'power_outage_mortatility'] == 'Sí' and df.loc[x, 'power_outage_deaths_count'] < 1:
        placholder_lst3.append(1)
       
    elif df.loc[x, 'mortality_hospital_failure_cardiovascular_cause'] == 'Sí' and df.loc[x, 'power_outage_deaths_count'] > 0:
            placholder_lst3.append(0)
    else:
        placholder_lst3.append(0)
    x += 1


In [17]:
# Imputing 'power_outage_equipment_failure_specify '
# Many values are filled as 0, already where no equipment failure occured or it is unknown.
# We impute 0 for missing values here as well. Very different way of handling this column
# For data accuracy it might be good to turn this field into a dropdown for the survey
fill = '0'
df['power_outage_equipment_failure_specify'] = df['power_outage_equipment_failure_specify'].fillna(fill)
df['power_outage_equipment_failure_specify'][df.power_outage_equipment_failure_specify == 'ninguno'] = "0"
df['power_outage_equipment_failure_specify'][df.power_outage_equipment_failure_specify == 'Ninguno'] = "0"
df['power_outage_equipment_failure_specify'][df.power_outage_equipment_failure_specify == 'NINGUNO'] = "0"
df['power_outage_equipment_failure_specify'][df.power_outage_equipment_failure_specify == 'No'] = "0"
df['power_outage_equipment_failure_specify'][df.power_outage_equipment_failure_specify == 'no hubo cortes de energia electrica'] = "0"
df['power_outage_equipment_failure_specify'][df.power_outage_equipment_failure_specify == 'No hubo falla'] = "0"


/Users/skanderdriss/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/skanderdriss/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/skanderdriss/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __na

In [18]:

x=0
placholder_lst4 =[]
while x < lenght1 :
    if df.loc[x, 'power_generator_available'] == 'Funciona todos los días' and df.loc[x, 'power_outage_equipment_failure_specify'] != '0':
        placholder_lst4.append(1)
       
    elif df.loc[x, 'power_generator_available'] == 'Funciona todos los días' and df.loc[x, 'power_outage_equipment_failure_specify'] != '00':
        placholder_lst4.append(1)
    else:
        placholder_lst4.append(0)
    x += 1


In [19]:
df6 = pd.DataFrame(placholder_lst)
df7 = pd.DataFrame(placholder_lst2)
df8 = pd.DataFrame(placholder_lst3)
df13 = pd.DataFrame(placholder_lst4)
df9 = (df6+df7+df8+df13)/4

In [22]:
df["logical_inconsistency"] = df9

In [21]:
# Creating #Strike_free colummn
# Creating a coulumn to see if there where any strikes by combining the missing value columns of the 
# different strike columns. If all values are missing there was no strike

df1 = []
df1 = (df['m_strike_other_affected'] + df['m_strike_patients_affected'] + df['m_strike_other_staff_affected'] 
+ df['m_strike_nurses_affected']+ df['m_strike_medical_staff_affected'])



no =[5]
yes=[0,1,2,3,4]
placeholder_lst = []

for strike in df1:
        if  strike in no:
            placeholder_lst.append(1)
            
        elif strike in yes:
            placeholder_lst.append(0)
        else:
            placeholder_lst.append(0)
        
df2 = pd.DataFrame(placeholder_lst)


# Concatenating with original DataFrame
df["strike_free"] = df2


KeyError: 'm_strike_other_affected'

In [23]:
df10 = df.loc[df.duplicated(['report_date', 'hospital_code'],keep='first'), ['timestamp','report_date', 'hospital_code']]
duplicate_list = df10['timestamp'].tolist()
duplicate_list
placeholder_lst3=[]

for x in df['timestamp']:
        if  x in duplicate_list:
            placeholder_lst3.append(1)
        else:
            placeholder_lst3.append(0)
              
df11 = pd.DataFrame(placeholder_lst3)


# Concatenating with original DataFrame
df['duplicates']=df11

Removing duplicate reports and calculating the total number of duplicates as well as the percentage of duplicates of total reports for the quality indicators.
Duplicates might be due to input errors, corrections or miscommunication between doctors. We decided to drop the older duplicates and only keep the most recent report in case it was sent in as a correction.

In [24]:
# Removing duplicates
later_duplicated_hospital(df)

,timestamp,report_week,operability_icu,power_outage_equipment_failure_specify,power_generator_available,power_outage_mortatility,power_outage_deaths_count,mortality_hospital_failure_cardiovascular_count,mortality_hospital_failure_cardiovascular_cause,mortality_hospital_failure_trauma_count,mortality_hospital_failure_trauma_cause,strike_medical_staff_affected,strike_nurses_affected,operability_icu_p,strike_other_staff_affected,strike_patients_affected,strike_other_affected,staff_violence_affected_reasons,epidemiological_emergency_suspected_diseases,rrt_avail_high_flow_catheters,rrt_avail_blood_tests_hiv_hvb_hvc_vdr,rrt_avail_immediate_access_urea_reduction_bun,nCoV_face_mask_avail,nCoV_respiratory_isolation_protocol_avail,operability_er,nCoV_isolation_area_avail,operability_sx,operability_lab,operability_uls,operability_ct_mri,operability_xr,er_avail_adrenalin,er_avail_atropine,hospital_code,er_avail_dopamine,er_avail_cephalosporins_betalactams,er_avail_aminoglycosides_quinolone,er_avail_vancomycin_clindamycin,er_avail_lidocaine,er_avail_minor_opioids,er_avail_major_opioids,er_avail_iv_fluids,er_avail_diazepam_dph,er_avail_heparin,federal_entity,er_avail_steroids,er_avail_insulin,er_avail_asthma,er_avail_blood_pressure,er_avail_defibrillator,er_avail_ott_intubation,er_avail_catheter,er_avail_oxygen_suction,sx_avail_minor_opioids,sx_avail_major_opioids,hospital_type,sx_avail_anesthetic_gases,sx_avail_anesthetics_iv,sx_avail_relaxants,sx_avail_ott_intubation,sx_avail_patient_lingerie_kit,sx_avail_disposables_mask_gloves_gown,sx_avail_oxygen_suction,rrt_avail,rrt_operability,rrt_date_stopped_operability,administrative_entity,rrt_reason_stopped_operability,rrt_avg_daily_patients,rrt_peritoneal_count,rrt_peritoneal_reason_not_performed,rrt_num_hemodialysis,rrt_num_hemodialysis_acute,rrt_num_hemodialysis_chronic,rrt_num_hemodialysis_equipments,rrt_num_hemodialysis_equipments_operability,rrt_hemodialysis_avail_filter,arch_beds_count,rrt_hemodialysis_avail_lines,rrt_hemodialysis_avail_kit_hemodialysis,rrt_hemodialysis_avail_iron,rrt_hemodialysis_avail_b_complex,rrt_hemodialysis_avail_calcium,rrt_hemodialysis_avail_zemblar,rrt_reverse_osmosis_unit_operability,rrt_staff_nephrology,rrt_staff_md,rrt_staff_resident,op_beds_count,rrt_staff_nurse,rrt_staff_nurse_nephrologist,nutr_avail,nutr_operability,nutr_date_stopped_operability,nutr_reason_stopped_operability,nutr_num,nutr_daily_freq_meal,nutr_quality,nutr_freq_milk_formulas,op_beds_er_count,pneumonia_antibiotic_therapy_latency,mi_thrombolytic_treatment_latency,er_staff_residents_and_rural_day_on_call,er_staff_specialist_day_on_call,er_staff_mic_day_on_call,er_staff_nurse_day_on_call,er_staff_non_professional_nurse_day_on_call,er_staff_residents_and_rural_night_on_call,er_staff_specialist_night_on_call,er_staff_mic_night_on_call,op_pavilions_count,er_staff_nurse_night_on_call,er_staff_non_professional_nurse_night_on_call,wash_failure_icu,wash_failure_er,wahs_failure_sx,power_outage,power_outage_avg_failures_per_day,power_outage_days_count,power_outage_avg_duration,power_outage_equipment_failure,m_power_outage_deaths_count,m_mortality_hospital_failure_cardiovascular_count,m_mortality_hospital_failure_trauma_count,m_rrt_avg_daily_patients,m_rrt_peritoneal_count,m_rrt_num_hemodialysis,m_rrt_num_hemodialysis_acute,m_rrt_num_hemodialysis_chronic,m_rrt_num_hemodialysis_equipments,m_rrt_num_hemodialysis_equipments_operability,report_date,logical_inconsistency,duplicates
0,2019-08-20T22:45:52.803Z,32 del 2019,Todos los días,,Funciona todos los días,No,0.0,2.0,"Equipos, Medicamentos, Recurso Humano, Resto e...",0.0,,,,Nunca ha existido,,,,,,,,,,,Todos los días,,Todos los días,Entre 3 y 5 días,No operativa,Nunca ha existido,No operativa,No hubo,No hubo,AMA000,No hubo,No hubo,No hubo,No hubo,Entre 1 y 2 días,No hubo,No hubo,No hubo,No hubo,No hubo,Amazonas,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,No hubo,Entre 1 y 2 días,No hubo,No hubo,Tipo II,Todos los días,Todos los días,Todos los días,Tod

In [25]:
# Comparing number of observations before and after duplication
removed_rows = lenght1 - len(df)
# Removed duplicates as a percentage of observations
duplicateperc = removed_rows/ lenght1

<strong>Imputing missing values</strong><br><br>
Analysing missing values and  imputing appropriate values

In [26]:
# Imputing 'strike_medical_staff_affected'
# This variable only has two answers Protests with temporary suspension or Protests without suspension
# We can therfore assume that the null values are those without protests
fill = 'Sin protestas'
df['strike_medical_staff_affected'] = df['strike_medical_staff_affected'].fillna(fill)

In [27]:
# Imputing 'strike_nurses_affected'
# This variable only has two answers Protests with temporary suspension or Protests without suspension
# We can therfore assume that the null values are those without protests
fill = 'Sin protestas'
df['strike_nurses_affected'] = df['strike_nurses_affected'].fillna(fill)

In [28]:
# Imputing 'strike_other_staff_affected'
# This variable only has three answers Protests with temporary suspension, Protests without suspension
# or Protests with complete stop of activities. We can therfore assume that the null values are those without protests
fill = 'Sin protestas'
df['strike_other_staff_affected'] = df['strike_other_staff_affected'].fillna(fill)

In [29]:
# Imputing 'strike_patients_affected'
# This variable only has three answers Protests with temporary suspension, Protests without suspension
# or Protests with complete stop of activities. We can therfore assume that the null values are those without protests
fill = 'Sin protestas'
df['strike_patients_affected'] = df['strike_patients_affected'].fillna(fill)

In [30]:
# Imputing 'strike_other_affected'
# This variable only has two answers Protests with temporary suspension or Protests without suspension
# We can therfore assume that the null values are those without protests
fill = 'Sin protestas'
df['strike_other_affected'] = df['strike_other_affected'].fillna(fill)

In [31]:
#Corona Virus not applicable before 23 Jan

for row in df["timestamp"]:
    if row <='2020-01-23T19:11:54.191Z':
        df["nCoV_face_mask_avail"] = df["nCoV_face_mask_avail"].fillna("Not Applicable")
        df["nCoV_isolation_area_avail"] = df["nCoV_isolation_area_avail"].fillna("Not Applicable")
        df["nCoV_respiratory_isolation_protocol_avail"] = df["nCoV_respiratory_isolation_protocol_avail"].fillna("Not Applicable")
        

In [32]:
# Imputing nutrition service availability
# Correcting incorrect data input in row 949
# inputing not applicable in related columns for nutrition service not available

df.loc[949,'nutr_avail'] = 'Sí'
df['nutr_operability'][df.nutr_avail == 'No'] = "Not applicable"
df['nutr_date_stopped_operability'][df.nutr_avail == 'No'] = "Not applicable"
df['nutr_reason_stopped_operability'][df.nutr_avail == 'No'] = "Not applicable"
df['nutr_num'][df.nutr_avail == 'No'] = "Not applicable"
df['nutr_daily_freq_meal'][df.nutr_avail == 'No'] = "Not applicable"
df['nutr_quality'][df.nutr_avail == 'No'] = "Not applicable"
df['nutr_freq_milk_formulas'][df.nutr_avail == 'No'] = "Not applicable"


/Users/skanderdriss/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/skanderdriss/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/skanderdriss/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users

In [33]:
# If the nutrition service is operational, then there would be no inputs for 
# "Date since the nutrition service was not operational" and "Reason the nutrion service is not operational"

df.loc[994,'nutr_operability'] = 'No'
df.loc[951,'nutr_operability'] = 'No'

df['nutr_date_stopped_operability'][df.nutr_operability == 'Sí'] = "Not applicable"
df['nutr_reason_stopped_operability'][df.nutr_operability == 'Sí'] = "Not applicable"


/Users/skanderdriss/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/skanderdriss/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
# If there is no dialysis service in a hospital, then the rest of the 
# dialysis-related rows would not be applicable
df['rrt_avail_immediate_access_urea_reduction_bun'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_operability'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_date_stopped_operability'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_reason_stopped_operability'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_avg_daily_patients'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_peritoneal_count'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_peritoneal_reason_not_performed'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_num_hemodialysis'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_num_hemodialysis_acute'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_num_hemodialysis_chronic'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_num_hemodialysis_equipments'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_reverse_osmosis_unit_operability'][df.rrt_avail == 'No'] = "Not applicable"


/Users/skanderdriss/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/skanderdriss/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/skanderdriss/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http

In [35]:
# If the dialysis service is operational, then there would be no inputs for 
# "Date since the dialysis service was not operational" and 
# "Reason the dialysis service is not operational"
df['rrt_date_stopped_operability'][df.rrt_operability  == 'Sí'] = "Not applicable"
df['rrt_reason_stopped_operability'][df.rrt_operability  == 'Sí'] = "Not applicable"

/Users/skanderdriss/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/skanderdriss/anaconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [36]:
# Filling the rest of the dialysis missing cells with "Unknown" because
# they cannot be accurately imputed
df['rrt_operability'] = df['rrt_operability'].fillna("Unknown")
df['rrt_date_stopped_operability'] = df['rrt_date_stopped_operability'].fillna("Unknown")
df['rrt_reason_stopped_operability'] = df['rrt_reason_stopped_operability'].fillna("Unknown")
df['rrt_avg_daily_patients'] = df['rrt_avg_daily_patients'].fillna("Unknown")
df['rrt_peritoneal_count'] = df['rrt_peritoneal_count'].fillna("Unknown")
df['rrt_peritoneal_reason_not_performed'] = df['rrt_peritoneal_reason_not_performed'].fillna("Unknown")
df['rrt_num_hemodialysis'] = df['rrt_num_hemodialysis'].fillna("Unknown")
df['rrt_num_hemodialysis_acute'] = df['rrt_num_hemodialysis_acute'].fillna("Unknown")
df['rrt_num_hemodialysis_chronic'] = df['rrt_num_hemodialysis_chronic'].fillna("Unknown")
df['rrt_num_hemodialysis_equipments'] = df['rrt_num_hemodialysis_equipments'].fillna("Unknown")
df['rrt_num_hemodialysis_equipments_operability'] = df['rrt_num_hemodialysis_equipments_operability'].fillna("Unknown")
df['rrt_reverse_osmosis_unit_operability'] = df['rrt_reverse_osmosis_unit_operability'].fillna("Unknown")



In [37]:
# Filling in the rest of the nutrition service data with "Missing" because 
# the values cannot be accurately imputed

df["nutr_num"] = df["nutr_num"].fillna("Missing")
df["nutr_daily_freq_meal"] = df["nutr_daily_freq_meal"].fillna("Missing")
df["nutr_quality"] = df["nutr_quality"].fillna("Missing")
df["nutr_freq_milk_formulas"] = df["nutr_freq_milk_formulas"].fillna("Missing")


<strong>Strikes</strong><br><br>
Strikes effect different groups differently. It might be that some groups are simply not affected 
by the strikes or that the effect is different on the different groups. We might need to create a column to see if there where any strikes in a given week by defining a column that combines the values for all strike columns into one to see if there are any protests or not.

In [38]:
# Imputing 'staff_violence_affected_reasons'
# This is a multiselection row without a count. We can assume that the blank fields are from weeks
# without incident  
fill = 'None reported'
df['staff_violence_affected_reasons'] = df['staff_violence_affected_reasons'].fillna(fill)

In [39]:
# Imputing 'epidemiological_emergency_suspected_diseases'
# This is a multiselection row without a count. We can assume that the blank fields are from weeks
# without incident  
fill = 'None reported'
df['epidemiological_emergency_suspected_diseases'] = df['epidemiological_emergency_suspected_diseases'].fillna(fill)

<strong>New columns</strong><br><br>
Creating additional columns to create insight

In [40]:
# Creating Number of missing values and percentage of potential missing values column as part of a
# data quality index per row.

df4 = []
df4 = (df['m_strike_other_affected'] + df['m_strike_patients_affected'] + df['m_strike_other_staff_affected'] 
+ df['m_strike_nurses_affected']+ df['m_strike_medical_staff_affected']+ df['m_nutr_freq_milk_formulas']
+ df['m_nutr_quality']+ df['m_nutr_daily_freq_meal']+ df['m_nutr_num']
+ df['m_nutr_reason_stopped_operability'] + df['m_nutr_date_stopped_operability']+ df['m_nutr_operability']
+ df['m_rrt_staff_nurse_nephrologist']+ df['m_rrt_staff_nurse']+ df['m_rrt_staff_resident'] 
+ df['m_rrt_staff_md']+ df['m_rrt_staff_nephrology']+ df['m_rrt_reverse_osmosis_unit_operability']
+ df['m_rrt_hemodialysis_avail_zemblar']+ df['m_rrt_hemodialysis_avail_calcium']+ df['m_rrt_hemodialysis_avail_b_complex']
+ df['m_rrt_hemodialysis_avail_iron']+ df['m_rrt_hemodialysis_avail_kit_hemodialysis']+ df['m_rrt_hemodialysis_avail_lines'] 
+ df['m_rrt_hemodialysis_avail_filter']+ df['m_rrt_num_hemodialysis_equipments_operability']+ df['m_rrt_num_hemodialysis_equipments']            
+ df['m_rrt_num_hemodialysis_chronic']+ df['m_rrt_num_hemodialysis_acute']+ df['m_rrt_num_hemodialysis']
+ df['m_rrt_peritoneal_reason_not_performed']+ df['m_rrt_peritoneal_count']+ df['m_rrt_avg_daily_patients']      
+ df['m_rrt_reason_stopped_operability']+ df['m_rrt_date_stopped_operability']+ df['m_rrt_operability']           
+ df['m_nCoV_isolation_area_avail']+ df['m_nCoV_respiratory_isolation_protocol_avail']+ df['m_nCoV_face_mask_avail']         
+ df['m_rrt_avail_immediate_access_urea_reduction_bun']+ df['m_rrt_avail_blood_tests_hiv_hvb_hvc_vdr']+ df['m_rrt_avail_high_flow_catheters']        
+ df['m_epidemiological_emergency_suspected_diseases']+ df['m_staff_violence_affected_reasons']+ df['m_mortality_hospital_failure_trauma_cause']   
+ df['m_mortality_hospital_failure_trauma_count']+ df['m_mortality_hospital_failure_cardiovascular_cause']+ df['m_mortality_hospital_failure_cardiovascular_count'] 
+ df['m_power_outage_deaths_count']+ df['m_power_outage_equipment_failure_specify'])
df5 = df4 /50

df["number_of_missing_values"] = df4
df["percentage_of_potential_missing_values"] = df5

KeyError: 'm_strike_other_affected'

In [ ]:
df['data_quality_index'] =(1-((2*df['duplicates']+df['logical_inconsistency']+3*df["percentage_of_potential_missing_values"])/6))

In [41]:
#Ordinal_encoding_supply
one_hot_supply = ['rrt_staff_nurse_nephrologist','rrt_staff_nurse', 'rrt_hemodialysis_avail_lines', 'rrt_hemodialysis_avail_kit_hemodialysis', 'rrt_hemodialysis_avail_iron', 'rrt_hemodialysis_avail_b_complex','rrt_hemodialysis_avail_calcium','rrt_hemodialysis_avail_zemblar','rrt_staff_nephrology','rrt_staff_md','rrt_staff_resident','er_avail_adrenalin','er_avail_atropine', 'er_avail_dopamine','er_avail_cephalosporins_betalactams','er_avail_aminoglycosides_quinolone', 'er_avail_vancomycin_clindamycin','er_avail_lidocaine','er_avail_minor_opioids','er_avail_major_opioids', 'er_avail_iv_fluids','er_avail_diazepam_dph','er_avail_heparin','er_avail_steroids','er_avail_insulin', 'er_avail_asthma','er_avail_blood_pressure','er_avail_defibrillator','er_avail_ott_intubation', 'er_avail_catheter','er_avail_oxygen_suction','sx_avail_minor_opioids','sx_avail_major_opioids','sx_avail_anesthetic_gases','sx_avail_anesthetics_iv','sx_avail_relaxants','sx_avail_ott_intubation', 'sx_avail_patient_lingerie_kit','sx_avail_disposables_mask_gloves_gown','sx_avail_oxygen_suction', 'rrt_avail_high_flow_catheters', 'rrt_avail_blood_tests_hiv_hvb_hvc_vdr', 'rrt_avail_immediate_access_urea_reduction_bun']
df [one_hot_supply] = df[one_hot_supply].replace({"Nunca ha existido": np.nan, 
    'No hubo': 0,
    'Entre 1 y 2 días': 0.33, 
    'Entre 3 y 5 días':0.66,
    'Todos los días':1})

#Ordinal_encoding_operability
one_hot_operability = ['operability_icu', 'operability_icu_p','operability_er','operability_sx','operability_lab', 'operability_uls','operability_ct_mri','operability_xr']
df [one_hot_operability] = df[one_hot_operability].replace({"Nunca ha existido": np.nan, 
    'No existe': 0,
    'No operativa': 0,
    'Menos de 3 de días':0.33, 
    'Entre 3 y 5 días':0.66,
    'Todos los días':1})

#Ordinal_encoding_operating_machines
one_hot_machines = ['power_generator_available']
df [one_hot_machines] = df[one_hot_machines].replace({"Nunca ha habido": np.nan, 
    'Hay pero no funciona': 0,
    'Funciona menos de 3 días': 0.33,
    'Funciona entre 3 y 5 días': 0.66, 
    'Funciona todos los días':1})

#Ordinal_encoding_encoding_washing_conditions
one_hot_wash = ['wash_failure_icu', 'wash_failure_er', 'wahs_failure_sx']
df [one_hot_wash] = df[one_hot_wash].replace({'No hubo agua ningún dia': 0, "NNo hubo agua ningún dia": 0, 
    '< 3 días, sin soporte alterno (cisternas)': 0.16,
    '< 3 días, con soporte alterno': 0.33,                                          
    '3 a 5 días , sin soporte  alterno': 0.50,
    '3 a 5 días, con soporte alterno':0.66,
    'Funciona entre 3 y 5 días': 0.83,
    'Hubo agua todos los días': 1})

#Ordinal_encoding_encoding_power_outage
power_out = ['power_outage_days_count']
df [power_out] = df[power_out].replace({"Menos de 3 días": 0.33, 
    'Entre 3 y 5 días': 0.66,
    'Todos los días': 1})    

#Ordinal_encoding_strikes
one_hot_strike = ['strike_medical_staff_affected', 'strike_nurses_affected','strike_other_affected']
df [one_hot_strike] = df[one_hot_strike].replace({ 
    'Sin protestas': 0,
    'Protestas  sin suspensión de actividades': 0.5,
    'Protestas con suspensión temporal  de actividades': 1})

one_hot_strike = ['strike_other_staff_affected','strike_patients_affected']
df [one_hot_strike] = df[one_hot_strike].replace({ 
    'Sin protestas': 0,
    'Protestas  sin suspensión de actividades': 0.33,
    'Protestas con suspensión temporal  de actividades': 0.66,
    'Huelga (suspendidas todas  las actividades)':1})


In [ ]:
df.to_csv('public_services.csv')